In [14]:
pip install easyocr pdf2image

In [15]:
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.12).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [16]:
!which pdfinfo

/usr/bin/pdfinfo


EASYOCR (PROBLEM WAS LEAVING SINGLE DIGIT)

In [17]:
import easyocr
from pdf2image import convert_from_path
from PIL import Image
import numpy as np

class EasyOCRExtractor:
    def __init__(self, pdf_path, languages=['en'], gpu=False):
        self.pdf_path = pdf_path
        self.reader = easyocr.Reader(languages, gpu=gpu)
        print("✓ EasyOCR initialized")

    def extract_all(self, dpi=500, poppler_path=None):
        print(f"Converting PDF to images (DPI: {dpi})...")

        if poppler_path:
            pages = convert_from_path(self.pdf_path, dpi=500, poppler_path=poppler_path)
        else:
            pages = convert_from_path(self.pdf_path, dpi=500)

        print(f"Processing {len(pages)} pages...")
        results = []

        for page_num, page_image in enumerate(pages, 1):
            text = self._extract_text_from_page(page_image)
            results.append({
                "page_number": page_num,
                "text": text
            })
            print(f"✓ Page {page_num} extracted")

        return results

    def _extract_text_from_page(self, image: Image):
        img_array = np.array(image)
        result = self.reader.readtext(img_array)

        # collect ONLY text content in reading order
        lines = [item[1] for item in result]

        return "\n".join(lines)


In [18]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print(f'User uploaded file "{fn}"')
  # Assuming the user uploads 'train_sample_1.pdf'
  # You might need to adjust the filename if it's different

Saving Sample Document 1.pdf to Sample Document 1.pdf
Saving train_sample_1.pdf to train_sample_1 (1).pdf
Saving SAmple Document 2.pdf to SAmple Document 2.pdf
Saving Sample Document 3.pdf to Sample Document 3.pdf
User uploaded file "Sample Document 1.pdf"
User uploaded file "train_sample_1 (1).pdf"
User uploaded file "SAmple Document 2.pdf"
User uploaded file "Sample Document 3.pdf"


In [ ]:
extractor = EasyOCRExtractor("/content/train_sample_1.pdf", languages=['en'], gpu=False)
pages = extractor.extract_all()

for p in pages:
    print("PAGE:", p["page_number"])
    print(p["text"])
    print("------------------------")

✓ EasyOCR initialized
Converting PDF to images (DPI: 500)...
Processing 2 pages...
✓ Page 1 extracted


In [ ]:
import easyocr

# initialise reader once
reader = easyocr.Reader(['en'], gpu=True)

# run OCR on an image
result = reader.readtext("/content/sample_3.png")

# extract only text
extracted_text = "\n".join([item[1] for item in result])

print(extracted_text)


In [ ]:
import easyocr

# initialise reader once
reader = easyocr.Reader(['en'], gpu=True)

# run OCR on an image
result = reader.readtext("/content/sample_1.png")

# extract only text
extracted_text = "\n".join([item[1] for item in result])

print(extracted_text)


In [ ]:
import easyocr

# initialise reader once
reader = easyocr.Reader(['en'], gpu=True)

# run OCR on an image
result = reader.readtext("/content/sample_2.png")

# extract only text
extracted_text = "\n".join([item[1] for item in result])

print(extracted_text)


In [ ]:
import easyocr
import cv2
import numpy as np

# load image
img_path = "/content/sample_2.png"
img = cv2.imread(img_path)

reader = easyocr.Reader(['en'], gpu=True)
results = reader.readtext(img)

boxes = []  # store normalized bounding boxes

# collect bounding boxes
for (bbox, text, conf) in results:
    x1, y1 = bbox[0]
    x2, y2 = bbox[2]
    boxes.append((int(x1), int(y1), int(x2), int(y2)))


# sort boxes by y (top to bottom)
boxes = sorted(boxes, key=lambda b: b[1])

# group into rows
rows = []
current = [boxes[0]]

for b in boxes[1:]:
    # threshold for same row (adjust if needed)
    if abs(b[1] - current[-1][1]) < 20:
        current.append(b)
    else:
        rows.append(current)
        current = [b]
rows.append(current)

# draw boxes around each "cell"
for row in rows:
    for (x1, y1, x2, y2) in row:
        cv2.rectangle(img, (x1, y1), (x2, y2), (0,255,0), 2)

cv2.imwrite("cells_detected.png", img)
print("saved as cells_detected.png")


In [ ]:
import easyocr
import cv2
import numpy as np

img_path = "/content/sample_2.png"
img = cv2.imread(img_path)

reader = easyocr.Reader(['en'], gpu=True)
results = reader.readtext(img)

boxes = []

# collect boxes
for (bbox, text, conf) in results:
    x1, y1 = bbox[0]
    x2, y2 = bbox[2]
    boxes.append([int(x1), int(y1), int(x2), int(y2), text])

# sort by x so columns come together
boxes = sorted(boxes, key=lambda b: b[0])

merged = []
used = set()
vertical_threshold = 1  # tweak if needed

for i, b1 in enumerate(boxes):
    if i in used:
        continue

    x1, y1, x2, y2, text1 = b1
    group = [b1]

    for j, b2 in enumerate(boxes[i+1:], start=i+1):
        xx1, yy1, xx2, yy2, text2 = b2

        # same column = strong horizontal overlap
        horizontal_overlap = min(x2, xx2) - max(x1, xx1)
        if horizontal_overlap > 10:
            # vertically close = stack to merge
            if abs(yy1 - y2) < vertical_threshold or abs(y1 - yy2) < vertical_threshold:
                group.append(b2)
                used.add(j)

    # merge group text & bbox
    xs = [g[0] for g in group] + [g[2] for g in group]
    ys = [g[1] for g in group] + [g[3] for g in group]

    merged_x1, merged_y1 = min(xs), min(ys)
    merged_x2, merged_y2 = max(xs), max(ys)
    merged_text = " ".join([g[4] for g in sorted(group, key=lambda x: x[1])])

    merged.append([merged_x1, merged_y1, merged_x2, merged_y2, merged_text])


# draw merged bounding boxes
for (x1, y1, x2, y2, text) in merged:
    cv2.rectangle(img, (x1, y1), (x2, y2), (0,255,0), 2)

cv2.imwrite("merged_cells.png", img)
print("saved as merged_cells.png")

# print merged text for confirmation
for box in merged:
    print(box[-1])


In [ ]:
def merge_vertical(boxes, threshold=1):
    merged = []
    used = set()

    # Sort by X (left to right)
    boxes = sorted(boxes, key=lambda b: b[0])

    for i, b1 in enumerate(boxes):
        if i in used:
            continue

        x1, y1, x2, y2, text1 = b1
        group = [b1]

        for j, b2 in enumerate(boxes[i+1:], start=i+1):
            xx1, yy1, xx2, yy2, text2 = b2

            # strong horizontal overlap (same column)
            horiz_overlap = min(x2, xx2) - max(x1, xx1)

            # should be same column
            if horiz_overlap > 10:
                # small vertical gap
                if abs(yy1 - y2) < threshold or abs(y1 - yy2) < threshold:
                    group.append(b2)
                    used.add(j)

        # merge group
        xs = [g[0] for g in group] + [g[2] for g in group]
        ys = [g[1] for g in group] + [g[3] for g in group]

        merged_x1 = min(xs)
        merged_y1 = min(ys)
        merged_x2 = max(xs)
        merged_y2 = max(ys)

        # sort text top → bottom, join with space
        merged_text = " ".join([g[4] for g in sorted(group, key=lambda t: t[1])])

        merged.append([merged_x1, merged_y1, merged_x2, merged_y2, merged_text])

    return merged


In [ ]:
def group_rows(boxes, row_threshold=18):
    # Sort top → bottom
    boxes = sorted(boxes, key=lambda b: b[1])

    rows = []
    current = [boxes[0]]

    for b in boxes[1:]:
        y = b[1]

        # compare with last element in current row
        last_y = current[-1][1]

        if abs(y - last_y) < row_threshold:
            current.append(b)
        else:
            rows.append(current)
            current = [b]

    rows.append(current)
    return rows


In [ ]:
def sort_row_cells(rows):
    clean = []
    for row in rows:
        clean.append(sorted(row, key=lambda b: b[0]))  # sort by x
    return clean


In [ ]:
import easyocr
import cv2

reader = easyocr.Reader(['en'], gpu=True)
img = cv2.imread("/content/sample_2.png")

raw = reader.readtext(img)

boxes = []
for (bbox, text, conf) in raw:
    x1, y1 = bbox[0]
    x2, y2 = bbox[2]
    boxes.append([int(x1), int(y1), int(x2), int(y2), text])

# step 1 — merge vertical
merged = merge_vertical(boxes)

# step 2 — group into table rows
rows = group_rows(merged)

# step 3 — reorder row cells left->right
table = sort_row_cells(rows)

# print table exactly as a grid
for row in table:
    print([cell[4] for cell in row])


In [ ]:
import easyocr
import cv2

img_path = "/content/sample_2.png"
img = cv2.imread(img_path)

reader = easyocr.Reader(['en'], gpu=False)
results = reader.readtext(img)

for (bbox, text, conf) in results:
    # bbox = 4 points: top-left, top-right, bottom-right, bottom-left
    pts = bbox

    # convert to integers
    pts = [(int(x), int(y)) for (x, y) in pts]

    # extract corners
    (x1, y1) = pts[0]   # top-left
    (x2, y2) = pts[2]   # bottom-right

    # FIX: ensure rectangle has thickness & valid coords
    # expand small boxes a little
    if x2 - x1 < 10:
        x2 += 2
        x1 -= 2
    if y2 - y1 < 10:
        y2 += 2
        y1 -= 2

    cv2.rectangle(img, (x1, y1), (x2, y2), (0,255,0), 2)

# save result
cv2.imwrite("debug_boxes.png", img)
print("Saved as debug_boxes.png")


In [ ]:
import easyocr
import cv2

img_path = "/content/sample_2.png"
reader = easyocr.Reader(['en'], gpu=True)

results = reader.readtext(img_path)

print("TOTAL DETECTIONS:", len(results))
for i, (bbox, text, conf) in enumerate(results):
    print(i, text, conf)


In [ ]:
img = cv2.imread("/content/sample_2.png")
img = cv2.resize(img, None, fx=2, fy=2, interpolation=cv2.INTER_LINEAR)
cv2.imwrite("/content/sample_2.png", img)


In [ ]:
import easyocr
import cv2
import matplotlib.pyplot as plt
import numpy as np

# image path
img_path = "/content/processed_output.png"

# OCR reader
reader = easyocr.Reader(['en'], gpu=True)

# run OCR
results = reader.readtext(
    img_path,
    detail=1,
    contrast_ths=0.1,
    adjust_contrast=0.7,
    min_size=3
)

# print detections
print("TOTAL DETECTIONS:", len(results))
for i, (bbox, text, conf) in enumerate(results):
    print(i, text, conf)

# -------------------------------
# DRAW BOUNDING BOXES ON IMAGE
# -------------------------------

img = cv2.imread(img_path)

for (bbox, text, conf) in results:
    pts = np.array(bbox).astype(int)

    # fix rectangle points
    x_min = pts[:,0].min()
    x_max = pts[:,0].max()
    y_min = pts[:,1].min()
    y_max = pts[:,1].max()

    # draw rectangle
    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0,255,0), 2)

# convert BGR → RGB for notebook
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# show output image
plt.figure(figsize=(10,10))
plt.imshow(img_rgb)
plt.axis('off')
plt.show()


In [ ]:
import cv2
import numpy as np

def preprocess(img):
    img = cv2.fastNlMeansDenoisingColored(img, None, 10, 10, 7, 21)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)

    th = cv2.adaptiveThreshold(
        gray, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY,
        31, 10
    )

    kernel = np.ones((2,2), np.uint8)
    thicken = cv2.dilate(th, kernel, iterations=1)

    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 40))
    remove_vertical = cv2.morphologyEx(thicken, cv2.MORPH_OPEN, vertical_kernel)

    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1))
    remove_horizontal = cv2.morphologyEx(thicken, cv2.MORPH_OPEN, horizontal_kernel)

    cleaned = thicken - remove_vertical - remove_horizontal

    cleaned = cv2.resize(cleaned, None, fx=2.0, fy=2.0, interpolation=cv2.INTER_CUBIC)

    return cleaned


In [ ]:
import cv2

img = cv2.imread("/content/sample_3.png")
preprocess(img)

In [ ]:
cv2.imwrite("processed_output.png", img)

TESSARACT WORKING WRONG HERE


In [ ]:
import cv2
import numpy as np
import pytesseract
from PIL import Image

def preprocess_scanned_image(image_path, dpi=300):
    # Read image
    img = cv2.imread(image_path)

    # 1. RESOLUTION ENHANCEMENT (if low DPI)
    height, width = img.shape[:2]
    if height < 2000:  # Low resolution check
        img = cv2.resize(img, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)

    # 2. GRAYSCALE CONVERSION
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 3. NOISE REMOVAL (crucial for scanned docs)
    denoised = cv2.medianBlur(gray, 3)
    # Alternative: Gaussian blur
    # denoised = cv2.GaussianBlur(gray, (3, 3), 0)

    # 4. CONTRAST ENHANCEMENT
    # CLAHE (Contrast Limited Adaptive Histogram Equalization)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    contrast_enhanced = clahe.apply(denoised)

    # 5. THRESHOLDING (binarization)
    # Global threshold
    _, thresh_global = cv2.threshold(contrast_enhanced, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Adaptive threshold (better for uneven lighting in scans)
    thresh_adaptive = cv2.adaptiveThreshold(contrast_enhanced, 255,
                                          cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                          cv2.THRESH_BINARY, 11, 2)

    # 6. DESKEWING (correct rotation)
    def deskew(image):
        coords = np.column_stack(np.where(image > 0))
        if len(coords) == 0:
            return image

        angle = cv2.minAreaRect(coords)[-1]
        if angle < -45:
            angle = -(90 + angle)
        else:
            angle = -angle

        (h, w) = image.shape[:2]
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC,
                               borderMode=cv2.BORDER_REPLICATE)
        return rotated

    deskewed = deskew(thresh_adaptive)

    # 7. MORPHOLOGICAL OPERATIONS (clean up text)
    kernel = np.ones((1, 1), np.uint8)
    cleaned = cv2.morphologyEx(deskewed, cv2.MORPH_CLOSE, kernel)
    cleaned = cv2.morphologyEx(cleaned, cv2.MORPH_OPEN, kernel)

    return cleaned, thresh_global, thresh_adaptive

def ocr_scanned_document(image_path):
    # Preprocess with multiple methods
    cleaned, global_thresh, adaptive_thresh = preprocess_scanned_image(image_path)

    # Try OCR on different preprocessed versions
    images_to_try = {
        'cleaned': cleaned,
        'global_thresh': global_thresh,
        'adaptive_thresh': adaptive_thresh
    }

    best_result = ""
    best_method = ""

    for method, img in images_to_try.items():
        # Tesseract config for scanned documents
        config = '--oem 3 --psm 6 -c tessedit_do_invert=0'

        text = pytesseract.image_to_string(img, config=config)

        # Simple heuristic: pick result with most valid content
        if len(text.strip()) > len(best_result.strip()):
            best_result = text
            best_method = method

    print(f"Best preprocessing method: {best_method}")
    return best_result

# Usage
result = ocr_scanned_document("/content/sample3rd.png")
print(result)

FINAL WORKING WITH TESSARACT

In [ ]:
# Step 1: Install Tesseract and dependencies in Colab
!sudo apt update
!sudo apt install tesseract-ocr
!sudo apt install libtesseract-dev

# Step 2: Install Python packages
!pip install pytesseract
!pip install opencv-python
!pip install Pillow

# Step 3: Import libraries
import cv2
import pytesseract
from PIL import Image
import numpy as np
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

# Step 4: Set tesseract path for Colab
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

# Step 5: Verify installation
print("Tesseract version:", pytesseract.get_tesseract_version())

In [ ]:
!pip install PyMuPDF

In [ ]:
import cv2
import numpy as np
import pytesseract
from PIL import Image
import fitz
import io

def simple_pdf_ocr(pdf_path, output_file="extracted_text.txt"):
    """
    Simple PDF to text extraction
    """
    # Convert PDF to images
    pdf_doc = fitz.open(pdf_path)
    all_text = {}

    for page_num in range(len(pdf_doc)):
        page = pdf_doc[page_num]
        # Convert to image
        pix = page.get_pixmap(matrix=fitz.Matrix(300/72, 300/72))
        img_data = pix.tobytes("ppm")
        img_pil = Image.open(io.BytesIO(img_data))
        img_cv = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

        # Preprocess
        gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # OCR
        text = pytesseract.image_to_string(thresh, config='--psm 6')
        all_text[page_num + 1] = text

        print(f"Page {page_num + 1} processed")

    pdf_doc.close()

    # Save to file
    with open(output_file, 'w', encoding='utf-8') as f:
        for page_num, text in all_text.items():
            f.write(f"\n{'='*60}\n")
            f.write(f"PAGE {page_num}\n")
            f.write(f"{'='*60}\n\n")
            f.write(text)
            f.write("\n")

    print(f"Text extracted to: {output_file}")
    return all_text

# Quick usage in Colab:

# uploaded = files.upload()
# pdf_file = list(uploaded.keys())[0]
result = simple_pdf_ocr('/content/train_sample_11.pdf')

In [ ]:
import re
import json

# QUICK TESTER - Reads from extracted.txt
def test_extraction():
    print("🔍 TESTING EXTRACTION FROM extracted.txt...")
    print("=" * 70)

    # Read the file
    with open('extracted_text.txt', 'r') as f:
        text = f.read()

    # Simple pattern matching for your bill format
    pattern = r'(\d+)\s+([A-Z\s]+?)\s+(\d+\.\d{2})\s+([\d,]+\.\d{2})\s+(\d+)\s+([\d,]+\.\d{2})'
    matches = re.findall(pattern, text)

    bill_items = []

    for match in matches:
        item_code, item_name, quantity, rate, discount, amount = match

        # Clean item name
        item_name = item_name.strip().replace('CHARGES', '').strip()

        bill_items.append({
            "item_name": item_name,
            "item_quantity": float(quantity),
            "item_rate": float(rate.replace(',', '')),
            "item_amount": float(amount.replace(',', ''))
        })

    # Display results
    print(f"📊 Found {len(bill_items)} bill items:")
    print("-" * 70)

    for i, item in enumerate(bill_items[:15], 1):  # Show first 15 items
        print(f"{i:2d}. {item['item_name']:25} | "
              f"Qty: {item['item_quantity']:4} | "
              f"Rate: {item['item_rate']:8.2f} | "
              f"Amount: {item['item_amount']:8.2f}")

    if len(bill_items) > 15:
        print(f"... and {len(bill_items) - 15} more items")

    print("=" * 70)
    print(f"💰 TOTAL AMOUNT: ₹{sum(item['item_amount'] for item in bill_items):,.2f}")

    return bill_items

# RUN THE TEST
result = test_extraction()

# Show JSON output for first 3 items
print("\n🎯 SAMPLE JSON OUTPUT (first 3 items):")
print(json.dumps({"bill_items": result[:3]}, indent=2))

PADDLEOCR (WORKING GOOD)

In [ ]:
!pip uninstall -y pillow
!pip install pillow==9.5.0

In [ ]:

# 对于 Linux 系统，执行：
!pip install https://paddle-whl.bj.bcebos.com/nightly/cu126/safetensors/safetensors-0.6.2.dev0-cp38-abi3-linux_x86_64.whl
# 对于Windows 系统，执行：

In [ ]:
# Uninstall conflicting packages
!pip uninstall paddleocr paddlex langchain langchain-community -y

# Install compatible versions
!pip install paddleocr==2.7.0.3
!pip install langchain==0.1.0
!pip install langchain-community

In [ ]:
!pip install paddlepaddle==3.2.1
!python -m pip install -U "paddleocr[doc-parser]"

In [ ]:
from paddleocr import PaddleOCR

# Initialize the object once
ocr = PaddleOCR(use_angle_cls=True, lang='en')




In [ ]:
def perform_ocr(image):
    result = ocr.ocr(image)
    print(f"Result type: {type(result)}")
    print(f"Result: {result}")
    return result

result = perform_ocr("/content/sample_2.png")

In [ ]:
if result and len(result) > 0:
    data = result[0]

    texts = data.get('rec_texts', [])
    scores = data.get('rec_scores', [])
    polys = data.get('rec_polys', [])

    print("="*60)
    print("EXTRACTED TEXT:")
    print("="*60)

    for idx, (text, score) in enumerate(zip(texts, scores)):
        print(f"{idx+1}. {text} (confidence: {score:.2%})")

    print("\n" + "="*60)
    print("ALL TEXT COMBINED:")
    print("="*60)
    print(' '.join(texts))


In [ ]:
result1 = perform_ocr("/content/sample_1.png")
if result1 and len(result1) > 0:
    data = result1[0]

    texts = data.get('rec_texts', [])
    scores = data.get('rec_scores', [])
    polys = data.get('rec_polys', [])

    print("="*60)
    print("EXTRACTED TEXT:")
    print("="*60)

    for idx, (text, score) in enumerate(zip(texts, scores)):
        print(f"{idx+1}. {text} (confidence: {score:.2%})")

    print("\n" + "="*60)
    print("ALL TEXT COMBINED:")
    print("="*60)
    print(' '.join(texts))


In [ ]:
result2 = perform_ocr("/content/sample3rd.png")
if result2 and len(result2) > 0:
    data = result2[0]

    texts = data.get('rec_texts', [])
    scores = data.get('rec_scores', [])
    polys = data.get('rec_polys', [])

    print("="*60)
    print("EXTRACTED TEXT:")
    print("="*60)

    for idx, (text, score) in enumerate(zip(texts, scores)):
        print(f"{idx+1}. {text} (confidence: {score:.2%})")

    print("\n" + "="*60)
    print("ALL TEXT COMBINED:")
    print("="*60)
    print(' '.join(texts))
